
## Train Real Gen Inverse Model 

* real data gen

## Web application 

Run interactively on the web at: 


## Inputs


* desired output requirements 

Raceway flame temp > 1900K.....(19, 'o_fta')
 
Hot metal temp > 1700K........(21, 'o_hmt')

Top gas temp > 100C,,,,,,,,,,,(23, 'o_tgt')

production rate ..............(39, 'o_prod_rt')


## Outputs



(2, 'i_h2i_rate')

(3, 'i_h2_temp')

(4, 'i_ngi_rate')

(6, 'i_pci_rate')

(11, 'i_o2_volfract')

(12, 'i_hbtemp')

(13, 'i_wind_rt')











## Installs


In [1]:

## !pip install xgboost
## !pip install onnxmltools
## !pip install onnxruntime
## !pip install pandas
## !pip install scikit-learn
## !pip install matplotlib
## !pip install seaborn
## !pip install mlxtend
## ! pip install skl2onnx



## Imports


In [2]:

import torch
import numpy as np
import pandas as pd
import sklearn
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import math
import seaborn as sns
import random
import functorch


import torch.optim as optim


from numpy.random import normal
from scipy.stats import norm
import scipy.stats as stats
from numpy import hstack
from numpy import vstack
from numpy import exp
from sklearn.neighbors import KernelDensity

##from typing import Optional

from torch import Tensor


from torch.utils.data import TensorDataset, DataLoader

from mlxtend.plotting import heatmap

## coefficient of determination (R**2)
from sklearn.metrics import r2_score

np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [3]:

import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import onnxmltools
from onnxmltools.convert.xgboost.operator_converters.XGBoost import (
    convert_xgboost)
from onnxmltools.convert import convert_xgboost as convert_xgboost_booster


from zipfile import ZipFile
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes)


# Import the ONNX runtime environment
import onnxruntime as rt



In [4]:

import PDFshapingUtils as PDF_tk



## Instantiate PDF shaping and regression class


In [5]:

PDFshapingOBJ = PDF_tk.PDFshapingUtils()



## Initialize parameters


In [6]:

PDFshapingOBJ.N_error_range = 5         ## 20
PDFshapingOBJ.mean_impulse  = 0.0       ## -7.0      ## 0.0
PDFshapingOBJ.std_impulse   = 0.001     ## 1.0
PDFshapingOBJ.kde_std       = 0.5       ## 0.1       ## 0.5      ## 3.0  ## smaller is better approximation?
PDFshapingOBJ.bandwidth     = 0.2       ## 0.2->more jagged, 2.0->more smooth


In [7]:

PDFshapingOBJ.initializeImpulseGaussian()



## Load Data


In [8]:

df_synthetic_data_for_inverse = PDFshapingOBJ.load_synth_folder_to_pandas("SyntheticCorpus", flatten=True)


In [9]:

PDFshapingOBJ.CFD_raw_data = df_synthetic_data_for_inverse 


In [10]:

PDFshapingOBJ.headers_list = PDFshapingOBJ.CFD_raw_data.columns.values.tolist()


In [11]:

PDFshapingOBJ.CFD_raw_data


,clamp_min_0,clamp_min_1,clamp_min_2,clamp_min_3,clamp_min_4,clamp_min_5,clamp_min_6,final_input_0,final_input_1,final_input_2,...,lower_bounds_glob_2,lower_bounds_glob_3,final_output_0,final_output_1,final_output_2,final_output_3,upper_bounds_glob_0,upper_bounds_glob_1,upper_bounds_glob_2,upper_bounds_glob_3
0,6.291000,0.000000,99.461998,41.283001,24.423000,1301.963989,195.013000,6.291000,0.000000,99.461998,...,77.660004,8857.799805,1969.712036,618.918030,77.660004,8857.799805,1969.712036,618.918030,77.660004,8857.799805
1,19.320999,461.717987,37.470001,23.632000,24.093000,1495.265991,186.029007,19.320999,461.717987,37.470001,...,58.433998,8320.955078,2248.246094,906.635986,58.433998,8320.955078,2248.246094,906.635986,58.433998,8320.955078
2,14.171000,0.000000,122.396004,0.000000,26.806000,1288.738037,180.975998,14.171000,0.000000,122.396004,...,66.882004,8721.851562,1953.932983,496.412994,66.882004,8721.851562,1953.932983,496.412994,66.882004,8721.851562
3,21.112000,549.929993,38.831001,0.000000,22.920000,1337.751953,154.199997,21.112000,549.929993,38.831001,...,49.710999,6908.180176,2097.268066,409.562988,49.710999,6908.180176,2097.268066,409.562988,49.710999,6908.180176
4,9.717000,76.620003,55.382000,0.000000,25.549999,1073.092041,229.688004,9.717000,76.620003,55.382000,...,106.440002,8757.669922,2047.034058,1225.357056,106.440002,8757.669922,2047.034058,1225.357056,106.440002,8757.669922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12890,17.587000,394.502014,45.917999,16.603001,24.193001,1392.828979,189.014999,17.587000,394.502014,45.917999,...,66.765999,8222.673828,2157.434082,872.961975,66.765999,8222.673828,2157.434082,872.961975,66.765999,8222.673828
12891,26.521000,279.338989,0.000000,28.233999,26.382999,1554.717041,153.210999,26.521000,279.338989,0.000000,...,42.397999,7016.518066,2323.868896,939.638977,42.397999,7016.518066,2323.868896,939.638977,42.397999,7016.518066
12892,0.000000,180.686005,60.108002,97.251999,25.070999,1385.245972,170.587997,0.000000,180.686005,60.108002,...,59.855999,7758.246094,2146.909912,896.197998,59.855999,7758.246094,2146.909912,896.197998,59.855999,7758.246094
12893,24.584000,285.221985,102.587997,7.750000,23.472000,1526.805054,231.763000,24.584000,285.221985,102.587997,...,75.721001,11193.183594,2071.056885,589.093994,75.721001,11193.183594,2071.056885,589.093994,75.721001,11193.183594


In [12]:

PDFshapingOBJ.print_headers_list()


['clamp_min_0', 'clamp_min_1', 'clamp_min_2', 'clamp_min_3', 'clamp_min_4', 'clamp_min_5', 'clamp_min_6', 'final_input_0', 'final_input_1', 'final_input_2', 'final_input_3', 'final_input_4', 'final_input_5', 'final_input_6', 'clamp_max_0', 'clamp_max_1', 'clamp_max_2', 'clamp_max_3', 'clamp_max_4', 'clamp_max_5', 'clamp_max_6', 'lower_bounds_glob_0', 'lower_bounds_glob_1', 'lower_bounds_glob_2', 'lower_bounds_glob_3', 'final_output_0', 'final_output_1', 'final_output_2', 'final_output_3', 'upper_bounds_glob_0', 'upper_bounds_glob_1', 'upper_bounds_glob_2', 'upper_bounds_glob_3']
33
(0, 'clamp_min_0')
(1, 'clamp_min_1')
(2, 'clamp_min_2')
(3, 'clamp_min_3')
(4, 'clamp_min_4')
(5, 'clamp_min_5')
(6, 'clamp_min_6')
(7, 'final_input_0')
(8, 'final_input_1')
(9, 'final_input_2')
(10, 'final_input_3')
(11, 'final_input_4')
(12, 'final_input_5')
(13, 'final_input_6')
(14, 'clamp_max_0')
(15, 'clamp_max_1')
(16, 'clamp_max_2')
(17, 'clamp_max_3')
(18, 'clamp_max_4')
(19, 'clamp_max_5')
(20, 'c

In [13]:
'''


PDFshapingOBJ.list_of_selected_column_names = ['i_h2i_rate','i_h2_temp','i_ngi_rate','i_ng_temp','i_pci_rate',
                    'i_wpi_rate','i_o2_volfract',
                    'i_hbtemp','i_wind_rt','o_prod_rt', 'o_tgt', 'o_hmt','o_fta']



PDFshapingOBJ.print_correlation_coefficients()

'''


"\n\n\nPDFshapingOBJ.list_of_selected_column_names = ['i_h2i_rate','i_h2_temp','i_ngi_rate','i_ng_temp','i_pci_rate',\n                    'i_wpi_rate','i_o2_volfract',\n                    'i_hbtemp','i_wind_rt','o_prod_rt', 'o_tgt', 'o_hmt','o_fta']\n\n\n\nPDFshapingOBJ.print_correlation_coefficients()\n\n"

In [14]:

PDFshapingOBJ.convert_pd_data_to_numpy()


[[    6.29099989     0.            99.46199799 ...   618.91802979
     77.66000366  8857.79980469]
 [   19.32099915   461.71798706    37.47000122 ...   906.63598633
     58.43399811  8320.95507812]
 [   14.17099953     0.           122.39600372 ...   496.41299438
     66.88200378  8721.8515625 ]
 ...
 [    0.           180.68600464    60.10800171 ...   896.19799805
     59.85599899  7758.24609375]
 [   24.58399963   285.22198486   102.58799744 ...   589.09399414
     75.72100067 11193.18359375]
 [   27.66699982   551.21099854     1.13399994 ...  1254.00695801
     26.91799927  6845.61376953]]
(12895, 33)



## Select inputs and outputs


In [15]:



'''
(21, 'lower_bounds_glob_0')
(22, 'lower_bounds_glob_1')
(23, 'lower_bounds_glob_2')
(24, 'lower_bounds_glob_3')

(29, 'upper_bounds_glob_0')
(30, 'upper_bounds_glob_1')
(31, 'upper_bounds_glob_2')
(32, 'upper_bounds_glob_3')
'''




PDFshapingOBJ.gen_X_y_for_selected_indeces(  
                   inputs = [ 25, 26, 27, 28 ] ,  
                   outputs= [  7, 8, 9, 10, 11, 12, 13 ]   
)


4
7


In [16]:

print(PDFshapingOBJ.X.shape)
print(PDFshapingOBJ.y.shape)


(12895, 4)
(12895, 7)


In [17]:

PDFshapingOBJ.random_seed = int( random.random() * 100  )         ## defautl is 42


In [18]:

PDFshapingOBJ.split_np_data_train_test(selected_test_size=0.2)


(10316, 4)
(2579, 4)
(10316, 7)
(2579, 7)


In [19]:

PDFshapingOBJ.convert_dataset_from_np_to_torch()


In [20]:

PDFshapingOBJ.standardize_X_scales()
PDFshapingOBJ.standardize_y_scales()


In [21]:

PDFshapingOBJ.gen_Dataloader_train()

PDFshapingOBJ.train_dl


In [22]:

n_inputs  = 4
n_outputs = 7

'''

class ResidualNet(nn.Module):
    def __init__(self, x_means, x_deviations, y_means, y_deviations, dropout_rate=0.1, device='cuda'):
        
        super().__init__()
        self.device = device
        
        self.x_means      = x_means.to(self.device)
        self.x_deviations = x_deviations.to(self.device)
        self.y_means      = y_means.to(self.device)
        self.y_deviations = y_deviations.to(self.device)

     
        self.fc1  = nn.Linear(n_inputs, 64)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(64, 64)
        self.act2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate)

        self.fc3 = nn.Linear(64, n_outputs)

        # If input_dim ≠ 64, project for residual addition
        self.input_proj = nn.Linear(n_inputs, 64) if n_inputs != 64 else nn.Identity()
        
        # Move model to device
        self.to(self.device)
        

    def forward(self, x):
        x  = x.to(self.device)
        x  = (x - self.x_means) / self.x_deviations
        x0 = self.input_proj(x)
        x1 = self.dropout1(self.act1(self.fc1(x)))
        x2 = self.dropout2(self.act2(self.fc2(x1)))
        x_res = x2 + x0
        y_scaled = self.fc3(x_res)
        y_descaled = y_scaled * self.y_deviations + self.y_means
        
        return y_descaled, y_scaled
    
'''


"\n\nclass ResidualNet(nn.Module):\n    def __init__(self, x_means, x_deviations, y_means, y_deviations, dropout_rate=0.1, device='cuda'):\n        \n        super().__init__()\n        self.device = device\n        \n        self.x_means      = x_means.to(self.device)\n        self.x_deviations = x_deviations.to(self.device)\n        self.y_means      = y_means.to(self.device)\n        self.y_deviations = y_deviations.to(self.device)\n\n     \n        self.fc1  = nn.Linear(n_inputs, 64)\n        self.act1 = nn.ReLU()\n        self.dropout1 = nn.Dropout(dropout_rate)\n\n        self.fc2 = nn.Linear(64, 64)\n        self.act2 = nn.ReLU()\n        self.dropout2 = nn.Dropout(dropout_rate)\n\n        self.fc3 = nn.Linear(64, n_outputs)\n\n        # If input_dim ≠ 64, project for residual addition\n        self.input_proj = nn.Linear(n_inputs, 64) if n_inputs != 64 else nn.Identity()\n        \n        # Move model to device\n        self.to(self.device)\n        \n\n    def forward(self, x

In [23]:



#############################################################
##   F1      plus       F2
## Linear     +      Nonlinear

class F1plusF2_SIO(nn.Module):
    ## initialize the layers
    def __init__(self, x_means, x_deviations, y_means, y_deviations,  device='cuda'):
        super().__init__()
        self.device = device
        
        ## self.x_means      = x_means
        ## self.x_deviations = x_deviations
        ## self.y_means      = y_means
        ## self.y_deviations = y_deviations
        
        
        self.x_means      = x_means.to(self.device)
        self.x_deviations = x_deviations.to(self.device)
        self.y_means      = y_means.to(self.device)
        self.y_deviations = y_deviations.to(self.device)
        
        
        
        ## F1
        self.f1_linear1 = nn.Linear(4, 7)       
        
        ## F2
        self.f2_linear1 = nn.Linear(4, 10)
        self.f2_act1    = nn.Sigmoid()                    ## Tanh()                       
        self.f2_linear2 = nn.Linear(10, 7)       
        self.f2_dropout = nn.Dropout(0.25)
        
        # Move model to device
        self.to(self.device)
        
        
    ## perform inference
    def forward(self, x):
        x = x.to(self.device)
        x = (x - self.x_means) / self.x_deviations
        
        ## F1
        f1 = self.f1_linear1(x)
        
        ## F2
        f2 = self.f2_linear1(x)
        f2 = self.f2_act1(f2)
        f2 = self.f2_dropout(f2)
        f2 = self.f2_linear2(f2)
        
        
        y_scaled   = f1 + f2
        y_descaled = y_scaled * self.y_deviations + self.y_means
        
        return y_descaled, y_scaled
    



In [24]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model     = F1plusF2_SIO(
             PDFshapingOBJ.x_means, 
             PDFshapingOBJ.x_deviations, 
             PDFshapingOBJ.y_means, 
             PDFshapingOBJ.y_deviations,
             device=device  # pass device into the model
    
)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn   = nn.MSELoss()




In [25]:

model.train()

for epoch in range(1000):
    for xb, yb in PDFshapingOBJ.train_dl:
        
        xb = xb.to(model.device)
        yb = yb.to(model.device)
        
        optimizer.zero_grad()
        pred_descaled, pred_scaled = model(xb)
        loss = loss_fn(pred_scaled, yb)
        loss.backward()
        optimizer.step()
    if epoch % 100 == 0:
        print(epoch, ".....", loss.item() ) 
    


    

0 ..... 0.5811542868614197
100 ..... 0.4097370505332947
200 ..... 0.36831799149513245
300 ..... 0.346495121717453
400 ..... 0.40355950593948364
500 ..... 0.5243855714797974
600 ..... 0.3336016833782196
700 ..... 0.4870888590812683
800 ..... 0.3608343005180359
900 ..... 0.5655982494354248


In [26]:

print("=== SANITY CHECK: SCALING ===")
print("y_means:", PDFshapingOBJ.y_means)
print("y_deviations:", PDFshapingOBJ.y_deviations)

print("===============================================")

print("x_means:", PDFshapingOBJ.x_means)
print("x_deviations:", PDFshapingOBJ.x_deviations)



=== SANITY CHECK: SCALING ===
y_means: tensor([[  14.6623,  305.5481,   45.1122,   36.6760,   25.5216, 1431.1283,
          190.2352]])
y_deviations: tensor([[ 12.6345, 281.6759,  36.4733,  39.6898,   3.3199, 160.6678,  25.8242]])
x_means: tensor([[2243.9016, 1039.0397,   63.8860, 8417.1729]])
x_deviations: tensor([[ 178.6109,  507.6813,   22.1074, 1175.1044]])



## Test model


In [27]:

print( PDFshapingOBJ.X_test_tr[0] )
print( PDFshapingOBJ.y_test_tr[0] )


tensor([2093.8120,  902.9690,   70.8630, 9296.1680])
tensor([  22.1480,    0.0000,   21.0880,   47.1270,   27.7620, 1164.4041,
         209.7550])


In [28]:


print( PDFshapingOBJ.X_test_tr.shape )
print( PDFshapingOBJ.y_test_tr.shape )

with torch.no_grad():
    
    pred_test_descaled, pred_test_scaled = model(  PDFshapingOBJ.X_test_tr  )


torch.Size([2579, 4])
torch.Size([2579, 7])


In [29]:


y_true = PDFshapingOBJ.y_test_tr.detach().cpu().numpy()

y_pred = pred_test_descaled.detach().cpu().numpy()  # or use descaled pair

mae  = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
rmse = mean_squared_error( y_true, y_pred, squared=False, multioutput='raw_values')
r2   = r2_score(           y_true, y_pred, multioutput='raw_values')

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)
print("Overall averages:", 
      mae.mean(), rmse.mean(), r2.mean())



MAE: [  4.9793825  135.3401      23.115265    31.551983     0.93298143
  99.60271      4.4458017 ]
RMSE: [  6.3632655 170.87119    28.450798   39.04742     1.2132748 123.85369
   5.622951 ]
R2: [0.74667367 0.63222894 0.3825443  0.07852167 0.86897658 0.35919006
 0.95033423]
Overall averages: 42.852604 53.631798 0.5740670640331175


In [30]:

r2   = r2_score(           y_true, y_pred, multioutput='raw_values')
r2


array([0.74667367, 0.63222894, 0.3825443 , 0.07852167, 0.86897658,
       0.35919006, 0.95033423])

In [31]:

r2.mean()


0.5740670640331175

In [32]:

print('Testing R**2 - descaled:', r2_score( y_pred, y_true   )  )


Testing R**2 - descaled: -1.5215673509291325


In [33]:

print('Testing R**2 - descaled:', r2_score( y_true,  y_pred   )  )


Testing R**2 - descaled: 0.5740670640331175


In [34]:

PDFshapingOBJ.r2_score_torch(PDFshapingOBJ.y_test_tr.to(PDFshapingOBJ.device), pred_test_descaled).mean()


tensor(0.5741, device='cuda:0')


## ONNX


In [35]:

import torch

# Your trained model
model.eval().float()


dummy_input = torch.randn(1, 4, dtype=torch.float32)

# Export to ONNX (single output called "output1")
torch.onnx.export(
    model,
    dummy_input,
    "ONNXmodels/F1F2_Inverse_HongRealGen.onnx",
    input_names=["input1"],
    output_names=["output1"],
    opset_version=15,              # fine for onnxruntime-web
    do_constant_folding=True,
    dynamic_axes={
        "input1": {0: "batch"},
        "output1": {0: "batch"}
    }
)
print("ONNX model saved")



ONNX model saved



## XGBoost


In [36]:

regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)


In [37]:

regressor.fit(PDFshapingOBJ.X_train, PDFshapingOBJ.y_train)


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [38]:

print( len(PDFshapingOBJ.X_train  ))
print( len(PDFshapingOBJ.y_train  ))


10316
10316


In [39]:

print( PDFshapingOBJ.X_train.shape  )
print( PDFshapingOBJ.y_train.shape  )


(10316, 4)
(10316, 7)


In [40]:

y_pred = regressor.predict(PDFshapingOBJ.X_test )


In [41]:

y_pred


array([[  10.661306  ,   68.70148   ,   76.43118   , ...,   26.433651  ,
        1331.4052    ,  204.98164   ],
       [   7.5912056 ,   54.32675   ,   74.4434    , ...,   29.210987  ,
        1371.3878    ,  189.6661    ],
       [  11.120284  ,  205.2797    ,   49.682922  , ...,   24.503937  ,
        1367.3735    ,  184.73325   ],
       ...,
       [   0.48936856,   17.630016  ,   91.86566   , ...,   24.110846  ,
        1401.0419    ,  207.52469   ],
       [  11.232839  ,  483.71317   ,   22.938517  , ...,   28.894894  ,
        1760.4896    ,  221.52014   ],
       [   2.7374516 ,   98.67716   ,   49.26787   , ...,   25.296757  ,
        1336.5511    ,  192.79828   ]], dtype=float32)

In [42]:

r2_score( PDFshapingOBJ.y_test, y_pred )


0.5705041565214447